In [1]:
!pip install -q findspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 43 kB/s 
     |████████████████████████████████| 199 kB 47.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=6046cc6a19dbab1589a44e445b769969b36bf0beb36c8a2ca5d0ebcf6b59ea6a
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sc.version

'3.3.1'

In [10]:
from pyspark.ml.regression import LinearRegression

# load training data

training = spark.read.format("libsvm").load("/content/drive/MyDrive/kuliah/Smester\ 5/big\ data\ &\ data\ mining/p12/sample_linear_regression_data.txt")


In [11]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [12]:
lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
# fit the model
lrModel = lr.fit(training)

In [14]:
# Print coefficients and intercept for linear regression
print("coefficient; %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))

coefficient; [0.0,0.3229251667740594,-0.3438548034562219,1.915601702345841,0.05288058680386255,0.765962720459771,0.0,-0.15105392669186676,-0.21587930360904645,0.2202536918881343]
Intercept: 0.15989368442397356


In [19]:
# Summarize the model over the training set and print out some metrics

trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

numIterations: 6
objectiveHistory: [0.49999999999999994, 0.4967620357443381, 0.49363616643404634, 0.4936351537897608, 0.4936351214177871, 0.49363512062528014, 0.4936351206216114]
+--------------------+
|           residuals|
+--------------------+
|  -9.889232683103197|
|  0.5533794340053553|
|  -5.204019455758822|
| -20.566686715507508|
|    -9.4497405180564|
|  -6.909112502719487|
|  -10.00431602969873|
|  2.0623978070504845|
|  3.1117508432954772|
|  -15.89360822941938|
|  -5.036284254673026|
|  6.4832158769943335|
|  12.429497299109002|
|  -20.32003219007654|
|    -2.0049838218725|
| -17.867901734183793|
|   7.646455887420495|
| -2.2653482182417406|
|-0.10308920436195645|
|  -1.380034070385301|
+--------------------+
only showing top 20 rows

RMSE: 10.189077
r2: 0.022861


In [20]:
from pyspark.ml.linalg import Vectors

df = spark.createDataFrame([
  (1.0, 2.0, Vectors.dense(1.0)),
  (0.0, 2.0, Vectors.sparse(1, [] , []))
  ], 
  ["label","weight", "features"] 
)

In [42]:
df.printSchema()

root
 |-- label: double (nullable = true)
 |-- weight: double (nullable = true)
 |-- features: vector (nullable = true)



In [43]:
df.show()

+-----+------+---------+
|label|weight| features|
+-----+------+---------+
|  1.0|   2.0|    [1.0]|
|  0.0|   2.0|(1,[],[])|
+-----+------+---------+



In [44]:
lr1 = LinearRegression(regParam=0.1, solver="normal", weightCol="weight")
lr1.setMaxIter(5)

LinearRegression_f3f8eb68ec8a

In [45]:
lr1.getMaxIter()

5

In [46]:
lr1.getRegParam()

0.1

In [56]:
model = lr1.fit(df)

In [57]:
model.setFeaturesCol("features")
model.setPredictionCol("newPrediction")

LinearRegressionModel: uid=LinearRegression_f3f8eb68ec8a, numFeatures=1

In [58]:
test0 = spark.createDataFrame([(Vectors.dense(-1.0),)], ["features"])

In [59]:
test0.show()

+--------+
|features|
+--------+
|  [-1.0]|
+--------+



In [60]:
model.intercept

0.08333333333333345

In [61]:
model.coefficients[0]

0.8333333333333333

In [62]:
model.predict(test0.head().features)

-0.7499999999999998

In [63]:
test1 = spark.createDataFrame([(Vectors.sparse(1, [0], [1.0]),)], ["features"])

In [64]:
model.transform(test1).head().newPrediction

0.9166666666666667